# 🎯 Construção da Pré-lista UNIVERSE_CANDIDATES

Este notebook executa o fluxo completo:
1. **Ingestão** de dados de mercado (Yahoo Finance)
2. **Pipeline** UNIVERSE_CANDIDATES (filtros e classificação)
3. **Resumo** da pré-lista gerada

---


## 1. Setup e Imports


In [25]:
# Instala dependências necessárias (execute apenas uma vez)
# Descomente a linha abaixo se pyarrow não estiver instalado
# !pip install pyarrow yfinance polars pyyaml --quiet

# Verifica se pyarrow está instalado
try:
    import pyarrow
    print(f"✅ pyarrow versão: {pyarrow.__version__}")
except ImportError:
    print("❌ pyarrow NÃO está instalado!")
    print("   Execute: pip install pyarrow")
    print("   Ou descomente a linha !pip install acima e re-execute esta célula")


✅ pyarrow versão: 22.0.0


In [ ]:
import sys
import os
import logging
from pathlib import Path
from datetime import datetime
import json

# Configura logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define o diretório raiz do projeto
# Detecta automaticamente se estamos em notebooks/ ou na raiz
current_dir = Path.cwd()
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
elif (current_dir / 'pyproject.toml').exists():
    project_root = current_dir
else:
    # Fallback: procura pyproject.toml subindo níveis
    project_root = current_dir
    for _ in range(5):
        if (project_root / 'pyproject.toml').exists():
            break
        project_root = project_root.parent

# Muda para o diretório do projeto (importante para paths relativos)
os.chdir(project_root)

# Adiciona módulos ao path
sys.path.insert(0, str(project_root / 'modules'))

print(f"📁 Project root: {project_root}")
print(f"📂 Working directory: {Path.cwd()}")
print(f"📅 Data/hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


📁 Project root: /home/wilson/PortfolioZero
📂 Working directory: /home/wilson/PortfolioZero
📅 Data/hora: 2025-12-02 17:20:50


In [ ]:
# Imports dos módulos do PortfolioZero
from portfoliozero.core.data.market_data_ingestion import (
    fetch_and_store_universe_market_data,
    validate_raw_market_data,
    load_data_source_config,
)

from portfoliozero.core.data.universe_candidates_pipeline import (
    build_universe_candidates,
    load_universe_candidates,
    validate_universe_candidates,
    get_pipeline_metadata,
)

import polars as pl

print("✅ Imports carregados com sucesso!")


✅ Imports carregados com sucesso!


## 2. Configuração

Carrega a configuração para ver os tickers que serão baixados.


In [ ]:
# Caminho do arquivo de configuração
config_path = str(project_root / 'config' / 'experiments' / 'universe_data_sources_v1.yaml')
print(f"📄 Config path: {config_path}")

# Carrega configuração
config = load_data_source_config(config_path)

print(f"\n📊 Provider: {config.get('provider', 'N/A')}")
print(f"📈 Tickers configurados: {len(config.get('universe', []))}")
print(f"📅 Período: {config.get('date_range', {}).get('start', 'N/A')} a {config.get('date_range', {}).get('end', 'N/A')}")
print(f"\n🏷️ Primeiros 10 tickers:")
for ticker in config.get('universe', [])[:10]:
    print(f"   - {ticker}")
if len(config.get('universe', [])) > 10:
    print(f"   ... e mais {len(config.get('universe', [])) - 10} tickers")


INFO - Configuração carregada: /home/wilson/PortfolioZero/config/experiments/universe_data_sources_v1.yaml
INFO -   Provider: yahoo_finance
INFO -   Tickers: 141


📄 Config path: /home/wilson/PortfolioZero/config/experiments/universe_data_sources_v1.yaml

📊 Provider: yahoo_finance
📈 Tickers configurados: 141
📅 Período: 2022-01-01 a today

🏷️ Primeiros 10 tickers:
   - ITUB4.SA
   - BBDC4.SA
   - BBAS3.SA
   - SANB11.SA
   - ITSA4.SA
   - B3SA3.SA
   - BBSE3.SA
   - BPAC11.SA
   - CIEL3.SA
   - WIZC3.SA
   ... e mais 131 tickers


## 3. Ingestão de Dados de Mercado 📥

Baixa dados OHLCV do Yahoo Finance para todos os tickers configurados.

⚠️ **Atenção:** Esta etapa pode demorar 5-10 minutos dependendo da quantidade de tickers e conexão.


In [29]:
# Define se deve sobrescrever dados existentes
from pickle import TRUE


OVERWRITE_EXISTING = True  # Mude para True se quiser re-baixar tudo

print("="*60)
print("INGESTÃO DE DADOS DE MERCADO")
print("="*60)
print(f"\n⚙️ Sobrescrever existentes: {OVERWRITE_EXISTING}")
print(f"🚀 Iniciando ingestão...\n")
# Executa ingestão (usa o config_path definido na célula anterior)
files = fetch_and_store_universe_market_data(config_path=config_path, overwrite=OVERWRITE_EXISTING)

print(f"\n✅ Ingestão concluída!")
print(f"📁 Arquivos gerados/atualizados: {len(files)}")


INFO - Configuração carregada: /home/wilson/PortfolioZero/config/experiments/universe_data_sources_v1.yaml
INFO -   Provider: yahoo_finance
INFO -   Tickers: 141
INFO - ============================================================
INFO - Iniciando ingestão de dados de mercado
INFO -   Provider: yahoo_finance
INFO -   Tickers: 141
INFO -   Período: 2022-01-01 a 2025-12-02
INFO - ============================================================
INFO - [1/141] Processando ITUB4.SA...


INGESTÃO DE DADOS DE MERCADO

⚙️ Sobrescrever existentes: True
🚀 Iniciando ingestão...



INFO -   ✓ Salvo: /home/wilson/PortfolioZero/data/raw/market/prices/ITUB4_SA.parquet (980 registros)
INFO - [2/141] Processando BBDC4.SA...
INFO -   ✓ Salvo: /home/wilson/PortfolioZero/data/raw/market/prices/BBDC4_SA.parquet (980 registros)
INFO - [3/141] Processando BBAS3.SA...
INFO -   ✓ Salvo: /home/wilson/PortfolioZero/data/raw/market/prices/BBAS3_SA.parquet (980 registros)
INFO - [4/141] Processando SANB11.SA...
INFO -   ✓ Salvo: /home/wilson/PortfolioZero/data/raw/market/prices/SANB11_SA.parquet (980 registros)
INFO - [5/141] Processando ITSA4.SA...
INFO -   ✓ Salvo: /home/wilson/PortfolioZero/data/raw/market/prices/ITSA4_SA.parquet (980 registros)
INFO - [6/141] Processando B3SA3.SA...
INFO -   ✓ Salvo: /home/wilson/PortfolioZero/data/raw/market/prices/B3SA3_SA.parquet (980 registros)
INFO - [7/141] Processando BBSE3.SA...
INFO -   ✓ Salvo: /home/wilson/PortfolioZero/data/raw/market/prices/BBSE3_SA.parquet (980 registros)
INFO - [8/141] Processando BPAC11.SA...
INFO -   ✓ Salvo:


✅ Ingestão concluída!
📁 Arquivos gerados/atualizados: 121


In [ ]:
# Valida dados brutos
validation = validate_raw_market_data()

print("\n📋 Validação dos dados brutos:")
print(f"   Válido: {'✅' if validation['valid'] else '❌'}")
print(f"   Arquivos: {validation.get('files', 0)}")
print(f"   Registros totais: {validation.get('total_records', 0):,}")
print(f"   Tickers únicos: {len(validation.get('tickers', []))}")

if validation.get('errors'):
    print(f"\n⚠️ Erros encontrados:")
    for err in validation['errors'][:5]:
        print(f"   - {err}")



📋 Validação dos dados brutos:
   Válido: ✅
   Arquivos: 121
   Registros totais: 118,342
   Tickers únicos: 121


## 4. Execução do Pipeline UNIVERSE_CANDIDATES 🔧

Aplica os filtros de seleção e gera a pré-lista de 60-80 candidatos.


In [ ]:
print("="*60)
print("PIPELINE UNIVERSE_CANDIDATES")
print("="*60)

# Executa pipeline
output_path = build_universe_candidates(force_refresh=True)

print(f"\n✅ Pipeline concluído!")
print(f"📁 Arquivo gerado: {output_path}")


INFO - ============================================================
INFO - Iniciando pipeline UNIVERSE_CANDIDATES
INFO - Project root: /home/wilson/PortfolioZero


PIPELINE UNIVERSE_CANDIDATES


INFO - ============================================================
INFO - Configurações carregadas com sucesso
INFO - ----------------------------------------
INFO - ETAPA 1: Leitura de dados brutos
INFO - Encontrados 121 arquivos Parquet em /home/wilson/PortfolioZero/data/raw/market
INFO - Dados brutos carregados: 118342 registros, 10 colunas
INFO - ----------------------------------------
INFO - ETAPA 2: Normalização de identificadores
INFO - Dados normalizados: 118342 registros
INFO - ----------------------------------------
INFO - ETAPA 3: Cálculo de métricas por ticker
INFO - Métricas calculadas para 121 tickers únicos
INFO - ----------------------------------------
INFO - ETAPA 4: Aplicação de filtros da pré-lista
INFO - Após filtro histórico mínimo: 121 ativos (removidos: 0)
INFO - Após filtro trading ratio: 121 ativos (removidos: 0)
INFO - Após filtro volume: 90 ativos (removidos: 31)
INFO - Após filtro preço: 83 ativos (removidos: 7)
INFO - Após filtro instrumentos: 83 ativos


✅ Pipeline concluído!
📁 Arquivo gerado: /home/wilson/PortfolioZero/data/universe/UNIVERSE_CANDIDATES.parquet


## 5. Carrega e Analisa Resultado 📊


In [ ]:
# Carrega resultado
df = load_universe_candidates(output_path)

# Valida
result = validate_universe_candidates(df)

print(f"📊 Candidatos carregados: {len(df)}")
print(f"✅ Validação: {'OK' if result.is_valid else 'FALHOU'}")
if result.warnings:
    print(f"⚠️ Warnings: {len(result.warnings)}")
    for w in result.warnings:
        print(f"   - {w}")


INFO - Carregado UNIVERSE_CANDIDATES: 68 registros de /home/wilson/PortfolioZero/data/universe/UNIVERSE_CANDIDATES.parquet


📊 Candidatos carregados: 68
✅ Validação: OK


In [ ]:
# Mostra primeiros registros
print("\n📋 Primeiros 15 candidatos (ordenados por volume):")
df.head(15)



📋 Primeiros 15 candidatos (ordenados por volume):


ticker,tipo_instrumento,setor,date_first,date_last,history_days,trading_days_ratio_252d,avg_volume_21d_brl,avg_price_recent_brl,last_price,annualized_vol_60d,volatility_class,liquidity_class
str,str,str,date,date,u32,f64,f64,f64,f64,f64,str,str
"""CSAN3.SA""","""ACAO_ON""","""Energia""",2022-01-03,2025-12-01,980,1.0,4.6898e7,6.092381,6.15,0.567496,"""ALTA""","""ALTA"""
"""PETR4.SA""","""ACAO_PN""","""Commodities""",2022-01-03,2025-12-01,980,1.0,4.3878e7,31.980477,31.85,0.179207,"""BAIXA""","""ALTA"""
"""B3SA3.SA""","""ACAO_ON""","""Financeiro""",2022-01-03,2025-12-01,980,1.0,3.7703e7,13.842381,14.89,0.25815,"""MEDIA""","""ALTA"""
"""ABEV3.SA""","""ACAO_ON""","""Consumo""",2022-01-03,2025-12-01,980,1.0,3.0933e7,13.484286,14.01,0.195297,"""BAIXA""","""ALTA"""
"""ITSA4.SA""","""ACAO_PN""","""Financeiro""",2022-01-03,2025-12-01,980,1.0,3.0450e7,11.90381,12.24,0.161098,"""BAIXA""","""ALTA"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""BEEF3.SA""","""ACAO_ON""","""Consumo""",2022-01-03,2025-12-01,980,1.0,2.0250e7,6.55381,6.05,0.414477,"""ALTA""","""ALTA"""
"""LREN3.SA""","""ACAO_ON""","""Consumo""",2022-01-03,2025-12-01,980,1.0,1.8988e7,15.066667,15.76,0.336896,"""MEDIA""","""ALTA"""
"""MGLU3.SA""","""ACAO_ON""","""Consumo""",2022-01-03,2025-12-01,980,1.0,1.7150e7,9.235238,10.34,0.578644,"""ALTA""","""ALTA"""


## 6. Resumo da Pré-lista 📈


In [ ]:
# Constantes
TARGET_MIN = 60
TARGET_MAX = 80

total = len(df)
in_range = TARGET_MIN <= total <= TARGET_MAX

print("="*60)
print("RESUMO DA PRÉ-LISTA UNIVERSE_CANDIDATES")
print("="*60)

print(f"\n📊 Total de candidatos: {total}")
print(f"🎯 Intervalo alvo: {TARGET_MIN} - {TARGET_MAX}")

if in_range:
    print(f"✅ Dentro do intervalo: SIM")
else:
    print(f"❌ Dentro do intervalo: NÃO")
    if total < TARGET_MIN:
        print(f"   ⚠️ Faltam {TARGET_MIN - total} candidatos para atingir o mínimo")
    else:
        print(f"   ⚠️ Excedem {total - TARGET_MAX} candidatos acima do máximo")


RESUMO DA PRÉ-LISTA UNIVERSE_CANDIDATES

📊 Total de candidatos: 68
🎯 Intervalo alvo: 60 - 80
✅ Dentro do intervalo: SIM


In [ ]:
# Distribuição por setor
if 'setor' in df.columns and len(df) > 0:
    print("\n📊 Distribuição por SETOR:")
    sector_counts = df.group_by('setor').len().sort('len', descending=True)
    for row in sector_counts.iter_rows():
        setor, count = row
        pct = (count / total * 100) if total > 0 else 0
        print(f"   {setor}: {count} ({pct:.1f}%)")



📊 Distribuição por SETOR:
   Consumo: 9 (13.2%)
   Energia: 9 (13.2%)
   Commodities: 9 (13.2%)
   Indústria: 9 (13.2%)
   Financeiro: 9 (13.2%)
   Tecnologia: 9 (13.2%)
   Utilidades: 5 (7.4%)
   Saúde: 5 (7.4%)
   Educação: 4 (5.9%)


In [ ]:
# Distribuição por volatilidade
if 'volatility_class' in df.columns and len(df) > 0:
    print("\n📊 Distribuição por VOLATILIDADE:")
    for vol_class in ['BAIXA', 'MEDIA', 'ALTA']:
        count = df.filter(pl.col('volatility_class') == vol_class).height
        pct = (count / total * 100) if total > 0 else 0
        print(f"   {vol_class}: {count} ({pct:.1f}%)")



📊 Distribuição por VOLATILIDADE:
   BAIXA: 15 (22.1%)
   MEDIA: 37 (54.4%)
   ALTA: 16 (23.5%)


In [ ]:
# Distribuição por liquidez
if 'liquidity_class' in df.columns and len(df) > 0:
    print("\n📊 Distribuição por LIQUIDEZ:")
    for liq_class in ['BAIXA', 'MEDIA', 'ALTA']:
        count = df.filter(pl.col('liquidity_class') == liq_class).height
        pct = (count / total * 100) if total > 0 else 0
        print(f"   {liq_class}: {count} ({pct:.1f}%)")



📊 Distribuição por LIQUIDEZ:
   BAIXA: 5 (7.4%)
   MEDIA: 24 (35.3%)
   ALTA: 39 (57.4%)


## 7. Metadados do Pipeline


In [ ]:
# Carrega metadados
metadata = get_pipeline_metadata(output_path)

if metadata:
    print("\n📋 Metadados do Pipeline:")
    print(f"   Execução: {metadata.execution_date}")
    print(f"   Registros entrada: {metadata.input_record_count:,}")
    print(f"   Registros saída: {metadata.output_record_count:,}")
    print(f"   Filtros aplicados: {len(metadata.filters_applied)}")
    
    print("\n   📌 Filtros:")
    for f in metadata.filters_applied:
        print(f"      - {f}")
    
    if metadata.warnings:
        print(f"\n   ⚠️ Warnings:")
        for w in metadata.warnings:
            print(f"      - {w}")



📋 Metadados do Pipeline:
   Execução: 2025-12-02 17:22:39.856157
   Registros entrada: 118,342
   Registros saída: 68
   Filtros aplicados: 6

   📌 Filtros:
      - min_history_days >= 252
      - trading_days_ratio >= 0.9
      - avg_volume_21d >= R$ 1,000,000
      - avg_price >= R$ 3.00
      - tipo_instrumento in ['ACAO_ON', 'ACAO_PN', 'BDR']
      - max_names_per_sector < 10

   ⚠️ Warnings:
      - Universo muito grande: 68 > 32 (máximo esperado)


## 8. Visualização dos Candidatos


In [ ]:
# Exibe tabela completa
print(f"\n📋 LISTA COMPLETA DE CANDIDATOS ({len(df)} ativos):")
df



📋 LISTA COMPLETA DE CANDIDATOS (68 ativos):


ticker,tipo_instrumento,setor,date_first,date_last,history_days,trading_days_ratio_252d,avg_volume_21d_brl,avg_price_recent_brl,last_price,annualized_vol_60d,volatility_class,liquidity_class
str,str,str,date,date,u32,f64,f64,f64,f64,f64,str,str
"""CSAN3.SA""","""ACAO_ON""","""Energia""",2022-01-03,2025-12-01,980,1.0,4.6898e7,6.092381,6.15,0.567496,"""ALTA""","""ALTA"""
"""PETR4.SA""","""ACAO_PN""","""Commodities""",2022-01-03,2025-12-01,980,1.0,4.3878e7,31.980477,31.85,0.179207,"""BAIXA""","""ALTA"""
"""B3SA3.SA""","""ACAO_ON""","""Financeiro""",2022-01-03,2025-12-01,980,1.0,3.7703e7,13.842381,14.89,0.25815,"""MEDIA""","""ALTA"""
"""ABEV3.SA""","""ACAO_ON""","""Consumo""",2022-01-03,2025-12-01,980,1.0,3.0933e7,13.484286,14.01,0.195297,"""BAIXA""","""ALTA"""
"""ITSA4.SA""","""ACAO_PN""","""Financeiro""",2022-01-03,2025-12-01,980,1.0,3.0450e7,11.90381,12.24,0.161098,"""BAIXA""","""ALTA"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""INBR32.SA""","""BDR""","""Tecnologia""",2022-08-26,2025-12-01,816,1.0,1.9685e6,48.547619,48.18,0.346729,"""MEDIA""","""BAIXA"""
"""CSMG3.SA""","""ACAO_ON""","""Utilidades""",2022-01-03,2025-12-01,980,1.0,1.8537e6,38.791429,42.77,0.217777,"""MEDIA""","""BAIXA"""
"""SAPR11.SA""","""ACAO_PN""","""Utilidades""",2022-01-03,2025-12-01,980,1.0,1.5027e6,35.485238,36.5,0.233732,"""MEDIA""","""BAIXA"""


---

## 🎯 Próximos Passos

### Se dentro do intervalo (60-80):
1. ✅ Revisar distribuição setorial
2. ✅ Revisar perfil de risco (volatilidade)
3. ➡️ Prosseguir para **TASK_011** (seleção dos 30 supervisionados)

### Se fora do intervalo:
1. Ajustar tickers em `config/experiments/universe_data_sources_v1.yaml`
2. Ajustar filtros em `config/experiments/universe_selection_rules_v1.yaml`
3. Re-executar este notebook

---

### Arquivos de Configuração:
- **Tickers:** `config/experiments/universe_data_sources_v1.yaml`
- **Filtros:** `config/experiments/universe_selection_rules_v1.yaml`

### Arquivos Gerados:
- **Pré-lista:** `data/universe/UNIVERSE_CANDIDATES.parquet`
- **CSV:** `data/universe/UNIVERSE_CANDIDATES.csv`
